In [1]:
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder


In [2]:
lung_data = pd.read_csv(r'C:\Users\elegb\Desktop\pdf\survey lung cancer.csv')
lung_data.head()

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
0,M,69,1,2,2,1,1,2,1,2,2,2,2,2,2,YES
1,M,74,2,1,1,1,2,2,2,1,1,1,2,2,2,YES
2,F,59,1,1,1,2,1,2,1,2,1,2,2,1,2,NO
3,M,63,2,2,2,1,1,1,1,1,2,1,1,2,2,NO
4,F,63,1,2,1,1,1,1,1,2,1,2,2,1,1,NO


In [3]:
lung_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309 entries, 0 to 308
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   GENDER                 309 non-null    object
 1   AGE                    309 non-null    int64 
 2   SMOKING                309 non-null    int64 
 3   YELLOW_FINGERS         309 non-null    int64 
 4   ANXIETY                309 non-null    int64 
 5   PEER_PRESSURE          309 non-null    int64 
 6   CHRONIC DISEASE        309 non-null    int64 
 7   FATIGUE                309 non-null    int64 
 8   ALLERGY                309 non-null    int64 
 9   WHEEZING               309 non-null    int64 
 10  ALCOHOL CONSUMING      309 non-null    int64 
 11  COUGHING               309 non-null    int64 
 12  SHORTNESS OF BREATH    309 non-null    int64 
 13  SWALLOWING DIFFICULTY  309 non-null    int64 
 14  CHEST PAIN             309 non-null    int64 
 15  LUNG_CANCER            

In [4]:
lung_data = lung_data.drop_duplicates()
lung_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 276 entries, 0 to 283
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   GENDER                 276 non-null    object
 1   AGE                    276 non-null    int64 
 2   SMOKING                276 non-null    int64 
 3   YELLOW_FINGERS         276 non-null    int64 
 4   ANXIETY                276 non-null    int64 
 5   PEER_PRESSURE          276 non-null    int64 
 6   CHRONIC DISEASE        276 non-null    int64 
 7   FATIGUE                276 non-null    int64 
 8   ALLERGY                276 non-null    int64 
 9   WHEEZING               276 non-null    int64 
 10  ALCOHOL CONSUMING      276 non-null    int64 
 11  COUGHING               276 non-null    int64 
 12  SHORTNESS OF BREATH    276 non-null    int64 
 13  SWALLOWING DIFFICULTY  276 non-null    int64 
 14  CHEST PAIN             276 non-null    int64 
 15  LUNG_CANCER            

In [5]:
categorical = lung_data.drop(['AGE'], axis = 1)
encoder = LabelEncoder()
for col in categorical.columns:
    categorical[col] = encoder.fit_transform(categorical[col])

categorical = categorical.astype("category")  
categorical.head()



,GENDER,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
0,1,0,1,1,0,0,1,0,1,1,1,1,1,1,1
1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,1
2,0,0,0,0,1,0,1,0,1,0,1,1,0,1,0
3,1,1,1,1,0,0,0,0,0,1,0,0,1,1,0
4,0,0,1,0,0,0,0,0,1,0,1,1,0,0,0


In [6]:
lung_data = pd.concat([categorical, lung_data['AGE']], axis = 1)
lung_data.head()

,GENDER,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER,AGE
0,1,0,1,1,0,0,1,0,1,1,1,1,1,1,1,69
1,1,1,0,0,0,1,1,1,0,0,0,1,1,1,1,74
2,0,0,0,0,1,0,1,0,1,0,1,1,0,1,0,59
3,1,1,1,1,0,0,0,0,0,1,0,0,1,1,0,63
4,0,0,1,0,0,0,0,0,1,0,1,1,0,0,0,63


In [7]:
lung_data.LUNG_CANCER.value_counts()

1    238
0     38
Name: LUNG_CANCER, dtype: int64

In [8]:
class_0 = lung_data[lung_data['LUNG_CANCER'] == 0]
class_1 = lung_data[lung_data['LUNG_CANCER'] == 1]
class_1 = class_1.sample(n = 500, replace = True)
class_0 = class_0.sample(n = 500, replace = True)
lung_data = pd.concat([class_0, class_1], axis = 0)
lung_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 183 to 87
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   GENDER                 1000 non-null   category
 1   SMOKING                1000 non-null   category
 2   YELLOW_FINGERS         1000 non-null   category
 3   ANXIETY                1000 non-null   category
 4   PEER_PRESSURE          1000 non-null   category
 5   CHRONIC DISEASE        1000 non-null   category
 6   FATIGUE                1000 non-null   category
 7   ALLERGY                1000 non-null   category
 8   WHEEZING               1000 non-null   category
 9   ALCOHOL CONSUMING      1000 non-null   category
 10  COUGHING               1000 non-null   category
 11  SHORTNESS OF BREATH    1000 non-null   category
 12  SWALLOWING DIFFICULTY  1000 non-null   category
 13  CHEST PAIN             1000 non-null   category
 14  LUNG_CANCER            1000 non-null   c

In [9]:
lung_data['LUNG_CANCER'].value_counts()

0    500
1    500
Name: LUNG_CANCER, dtype: int64

In [10]:
lung_data.head()

,GENDER,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER,AGE
183,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,71
4,0,0,1,0,0,0,0,0,1,0,1,1,0,0,0,63
37,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,56
14,1,1,0,0,0,0,0,1,1,1,1,0,0,1,0,69
8,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,68


In [11]:
X = lung_data.drop('LUNG_CANCER', axis =1)
y = lung_data.LUNG_CANCER

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size =0.15, random_state = 42)

print(y_test.value_counts())

1    80
0    70
Name: LUNG_CANCER, dtype: int64


In [13]:
categorical_columns = ['GENDER', 'SMOKING', 'YELLOW_FINGERS', 'ANXIETY', 'PEER_PRESSURE',
    'CHRONIC DISEASE', 'FATIGUE ', 'ALLERGY ', 'WHEEZING',
    'ALCOHOL CONSUMING', 'COUGHING', 'SHORTNESS OF BREATH',
    'SWALLOWING DIFFICULTY', 'CHEST PAIN',
]

In [14]:
# create DMatrix for training and test sets with categorical features enabled
X_train = X_train.astype({col: "category" for col in categorical_columns})

dtrain = xgb.DMatrix(X_train , label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, enable_categorical=True)

# set parameters for XGBoost classifier

params = {
    'objective': 'binary:logistic',
    'max_depth':3, 
    'eta':1, 
    'nthread': 3,
    'eval_metric': 'auc',
    'learning_rate': 1
}

# train model
model = xgb.train(params, dtrain, num_boost_round=100)
# make predictions on test data
y_pred = model.predict(dtest)

# convert probabilities to binary predictions
y_pred_binary = [1 if p >= 0.99 else 0 for p in y_pred]

# evaluate model performance
print(classification_report(y_test, y_pred_binary))
confusion_matrix(y_test, y_pred_binary)

              precision    recall  f1-score   support

           0       0.95      1.00      0.97        70
           1       1.00      0.95      0.97        80

    accuracy                           0.97       150
   macro avg       0.97      0.97      0.97       150
weighted avg       0.97      0.97      0.97       150



array([[70,  0],
       [ 4, 76]], dtype=int64)

#### Lets assume i want to make predictions  with that of a new patient coming in

In [16]:
input_dict = {'GENDER': 1, 'SMOKING': 0, 'YELLOW_FINGERS':0,
                  'ANXIETY': 1, 'PEER_PRESSURE': 0,
                  'CHRONIC DISEASE': 0, 'FATIGUE ': 0,
                  'ALLERGY ': 0, 'WHEEZING': 0,
                  'ALCOHOL CONSUMING': 0, 'COUGHING': 1,
                  'SHORTNESS OF BREATH': 0,
                  'SWALLOWING DIFFICULTY': 0,
                  'CHEST PAIN': 1, 'AGE': 25}
input_df = pd.DataFrame.from_dict([input_dict])
input_df.astype({col: "category" for col in categorical_columns})
input_df

,GENDER,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,AGE
0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,25


In [17]:
input_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   GENDER                 1 non-null      int64
 1   SMOKING                1 non-null      int64
 2   YELLOW_FINGERS         1 non-null      int64
 3   ANXIETY                1 non-null      int64
 4   PEER_PRESSURE          1 non-null      int64
 5   CHRONIC DISEASE        1 non-null      int64
 6   FATIGUE                1 non-null      int64
 7   ALLERGY                1 non-null      int64
 8   WHEEZING               1 non-null      int64
 9   ALCOHOL CONSUMING      1 non-null      int64
 10  COUGHING               1 non-null      int64
 11  SHORTNESS OF BREATH    1 non-null      int64
 12  SWALLOWING DIFFICULTY  1 non-null      int64
 13  CHEST PAIN             1 non-null      int64
 14  AGE                    1 non-null      int64
dtypes: int64(15)
memory usage: 248.0 bytes


In [18]:
categorical_columns = ['GENDER', 'SMOKING', 'YELLOW_FINGERS', 'ANXIETY', 'PEER_PRESSURE',
    'CHRONIC DISEASE', 'FATIGUE ', 'ALLERGY ', 'WHEEZING',
    'ALCOHOL CONSUMING', 'COUGHING', 'SHORTNESS OF BREATH',
    'SWALLOWING DIFFICULTY', 'CHEST PAIN',
]


dtest = xgb.DMatrix(input_df)dte

prediction = model.predict(dtest)
prediction

array([0.20313403], dtype=float32)

In [19]:
import gradio as gr

# Define the Gradio input and output interfaces
inputs = [
    gr.inputs.Dropdown(choices=["0", "1"], label="Gender"),
    gr.inputs.Dropdown(choices=["0", "1"], label="Do you smoke?"),
    gr.inputs.Dropdown(choices=["0", "1"], label="Do you have Yellow Fingers"),
    gr.inputs.Dropdown(choices=["0", "1"], label="Do you have Anxiety"),
    gr.inputs.Dropdown(choices=["0", "1"], label="Do you get influenced by Peer Pressure"),
    gr.inputs.Dropdown(choices=["0", "1"], label="Do you have any Chronic Disease"),
    gr.inputs.Dropdown(choices=["0", "1"], label="Do you have Fatigue"),
    gr.inputs.Dropdown(choices=["0", "1"], label="Do you have an Allergy"),
    gr.inputs.Dropdown(choices=["0", "1"], label="Do you experience Wheezing"),
    gr.inputs.Dropdown(choices=["0", "1"], label="Do you drink alcohol"),
    gr.inputs.Dropdown(choices=["0", "1"], label="Are you Coughing"),
    gr.inputs.Dropdown(choices=["0", "1"], label="Do you have Shortness of Breath"),
    gr.inputs.Dropdown(choices=["0", "1"], label="Do you have Swallowing Difficulty"),
    gr.inputs.Dropdown(choices=["0", "1"], label="Do you have Chest Pain"),
    gr.inputs.Number(label='Age')
]

output = gr.outputs.Label(num_top_classes=2)

# Define the predict function
def predict(gender, smoking, yellow_fingers, anxiety, peer_pressure,
            chronic_disease, fatigue, allergy, wheezing, alcohol_consuming,
            coughing, shortness_of_breath, swallowing_difficulty, chest_pain,
            age):
    # Create a dataframe with the input values
    input_dict = {'GENDER': gender, 'SMOKING': smoking, 'YELLOW_FINGERS': yellow_fingers,
                  'ANXIETY': anxiety, 'PEER_PRESSURE': peer_pressure,
                  'CHRONIC DISEASE': chronic_disease, 'FATIGUE ': fatigue,
                  'ALLERGY ': allergy, 'WHEEZING': wheezing,
                  'ALCOHOL CONSUMING': alcohol_consuming, 'COUGHING': coughing,
                  'SHORTNESS OF BREATH': shortness_of_breath,
                  'SWALLOWING DIFFICULTY': swallowing_difficulty,
                  'CHEST PAIN': chest_pain, 'AGE': age}
    input_df = pd.DataFrame.from_dict([input_dict]).astype("int")
    
    dtest = xgb.DMatrix(input_df)
    
    
    #make predictions
    prediction = model.predict(dtest)
    
    # Return prediction
    return "You have Lung Cancer, you might want to see the Doctor." if prediction >0.99 else "You don't have Lung Cancer, Enjoy❤"

# Create and launch the interface
interface = gr.Interface(fn=predict, inputs=inputs, outputs=output, 
                     title='Lung Cancer Prediction', description='Predicting lung cancer using XGBoost Classifier.\nPlease Note:\nFemale = 0, Male= 1\nNo = 0, Yes = 1')
interface.launch(auth = ('user', 'atom'), share = True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://de43ecf59e54e5afd9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


In [25]:
import joblib
# save model
joblib.dump(model, "model.pkl")

['model.pkl']

In [26]:
#lets load the model to make predictions
model = joblib.load("model.pkl")
model.predict(dtest)

array([0.20313403], dtype=float32)